In [4]:
import pandas as pd
import numpy as np

from sklearn.ensemble import (AdaBoostClassifier, GradientBoostingClassifier,
                              RandomForestClassifier, ExtraTreesClassifier)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.datasets import load_digits

from tqdm import tqdm

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats.distributions import randint

Разберём стекинг на практике, проанализировав датасет, описывающий параметры, которые были сняты со спутника при фотографировании Земли. У нас есть 54 переменные. Для упрощения будем рассматривать два типа поверхностей —так мы сводим нашу задачу к задаче бинарной классификации. Сделаем базовую предобработку, воспользуемся StandardScaler.

Стекинг позволяет объединять ответы нескольких алгоритмов первого уровня в один большой ответ при помощи нового алгоритма обучения. Для избежания переобучения будем разбивать обучающую выбоку на фолды — разбиение обучающей выборки на несколько частей. На каждом фолде мы обучаем алгоритм заново.

Функция compute_meta_feature принимает на вход один алгоритм и возвращает новые признаки на объектах, которые не использовались во время обучения. Функция generate_meta_feature делает подобное, но принимает на вход несколько классификаторов в списке и повторяет процедуру, а затем генерирует и возвращает матрицу с изначальным количеством объектов, признаков будет столько, сколько мы передали классификаторов. 

In [5]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    
    X_meta_train = np.zeros_like(y_train, dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]
        
        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)
        X_meta_train[predict_fold_index] = folded_clf.predict_proba(X_fold_predict)[:, 1]
    
    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)
    
    X_meta_test = meta_clf.predict_proba(X_test)[:, 1]
    
    return X_meta_train, X_meta_test

def generate_meta_features(classifiers, X_train, X_test, y_train, cv):
   
    features = [
        compute_meta_feature(clf, X_train, X_test, y_train, cv)
        for clf in tqdm(classifiers)
    ]
    
    stacked_features_train = np.vstack([
        features_train for features_train, features_test in features
    ]).T

    stacked_features_test = np.vstack([
        features_test for features_train, features_test in features
    ]).T
    
    return stacked_features_train, stacked_features_test

Обучим один градиентный бустинг, в котором будем использовать 300 алгоритмов. Мы получим accuracy = 0.7899929527836504. Если мы будем агрегировать бустинг со случайным лесом и с двумя логистическими регрессиями, то получим результат лучше: 0.8118393234672304.



In [6]:
dataset = load_digits()
X, y = dataset['data'], dataset['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

Задание 6.6.1
1 point possible (graded)
В скринкасте мы разобрали схему генерации признаков в стекинге, когда для тестовой выборки алгоритм заново переобучался на всей тренировочной выборке. Реализуйте схему, когда вместо этого производится агрегация ответов всех обученных на фолдах классификаторов на тестовой выборке при помощи усреднения.

Логика решения:
1) Создадим X_meta_test, заполним его нулями (по аналогии с X_meta_train);
2) Далее на каждом шаге, где мы обучаем folded_clf.fit (X_fold_train, y_fold_train) и его предсказания на X_fold_predict запихиваем в X_meta_train[predict_fold_index] добавим еще одну строку, где в X_meta_test будем добавлять предсказания вероятностей folded_clf на X_test. Их можно сразу складывать друг с другом или сохранить много массивов, тогда в конце их нужно будет все сложить, а потом делить на количество сплитов (количество массивов равно количеству сплитов в кросс - валидации);
3) После цикла останется только усреднить все эти массивы, это и будет наш X_meta_test.
За основу нужно взять следующий код:

In [ ]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv):
    """    Эта функция подсчитывает признаки для мета-классификатора.    
    Они являются вероятностями классов при решении задачи многоклассовой классификации. 
    :arg clf: классификатор    :args X_train, y_train: обучающая выборка   
    :arg X_test: признаки тестовой выборки    :arg cv: класс, генерирующий фолды (KFold)    
    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок    """
    n_classes = len(np.unique(y_train))
    X_meta_train = np.zeros((len(X_train), n_classes), dtype=np.float32)
    for train_fold_index, predict_fold_index in cv.split(X_train):
        X_fold_train, X_fold_predict = X_train[train_fold_index], X_train[predict_fold_index]
        y_fold_train = y_train[train_fold_index]

        folded_clf = clone(clf)
        folded_clf.fit(X_fold_train, y_fold_train)

        X_meta_train[predict_fold_index] = folded_clf.predict_proba(
            X_fold_predict)

    meta_clf = clone(clf)
    meta_clf.fit(X_train, y_train)

    X_meta_test = meta_clf.predict_proba(X_test)

    return X_meta_train, X_meta_test

In [ ]:
def compute_meta_feature_mean(clf, X_train, X_test, y_train, cv):
    """
    Эта функция подсчитывает признаки для мета-классификатора. 
    Они являются вероятностями классов при решении задачи многоклассовой классификации.

    :arg clf: классификатор
    :args X_train, y_train: обучающая выборка
    :arg X_test: признаки тестовой выборки
    :arg cv: класс, генерирующий фолды (KFold)

    :returns X_meta_train, X_meta_test: новые признаки для обучающей и тестовой выборок
    """
# Напишите ваш код ниже